# Notebook dedicated to getting water mask using isce and mintpy
- https://github.com/insarlab/MintPy-tutorial/blob/main/applications/water_mask.ipynb

In [ ]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
from mintpy.cli import view
import re

# Something interesting I could try:
Since I am getting open water, subqeuous, and subaerial classes via temporally registered Sentinel-1 and Sentinel-2 pairs, I could use this labels as their own water mask files. Would be much finer resolution, and could potentially be more accurate. Would need to use geo2rdr to convert similar to below. For now going to stick with this method until everything is up and running for the most part.

# Retrieve the aoi from the .txt created in 01_get_slc

In [ ]:
isce_aoi_file = open('isceaoi.txt', 'r+')
aoi = isce_aoi_file.read()
aoi_list = [float(num) for num in re.findall(r'[-+]?\d*\.\d+', aoi)]

In [ ]:
os.chdir('/home/wcc/SabineRS_InSAR')

In [ ]:
!/home/wcc/tools/miniforge/envs/mintpy/lib/python3.8/site-packages/isce/applications/wbd.py {aoi_list[0]} {aoi_list[1]} {aoi_list[2]} {aoi_list[3]}
wbd_file = [f for f in os.listdir(os.getcwd()) if f.endswith('.wbd')][0]
view.main(f'{wbd_file} -c gray --noverbose'.split())

# convert to radar coordinates
# need to fix the .rdr files for lat and lon, i think
!geocode.py $wbd_file -o waterBody.rdr --lat-file /home/wcc/SabineRS_InSAR/interferometry/work/merged/lat.rdr --lon-file /home/wcc/SabineRS_InSAR/interferometry/work/merged/lon.rdr --geo2radar --fill 255
view.main('waterBody.rdr -c gray --flip-lr --noverbose'.split())

# adjust for mintpy
!generate_mask.py waterBody.rdr --max 0.5 -o waterMask.rdr
view.main('waterMask.rdr -c gray --flip-lr --noverbose'.split())

# Multilook water mask to same resolution as the end product ifg
- i am starting with 1x5 (azi x range) for finest resolution from S1